In [1]:
! pip install git+https://github.com/facebookresearch/segment-anything.git

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-k4u_u389
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-k4u_u389
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'segment_anything' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'segment_anything'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36636 sha256=c5ce2bbd9144a479d360ce0

In [2]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2025-10-27 10:10:34--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.161.104.79, 3.161.104.22, 3.161.104.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.161.104.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G  10.0MB/s    in 4m 15s  

2025-10-27 10:14:49 (9.59 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [15]:
! pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 3.8 MB/s  0:00:17 eta 0:00:010:00:01


In [29]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator

In [27]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [20]:
IMAGE_FOLDER = "solar-dataset"
MODEL_CHECKPOINT = "sam_vit_h_4b8939.pth"
MODEL_TYPE = "vit_h"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
print("Loading SAM model...")
sam = sam_model_registry[MODEL_TYPE](checkpoint=MODEL_CHECKPOINT)
sam.to(device=DEVICE)

predictor = SamPredictor(sam)

Loading SAM model...


In [23]:
import os
dataset = os.listdir(IMAGE_FOLDER)[:1]

In [30]:
mask_generator = SamAutomaticMaskGenerator(sam)

In [25]:
OUTPUT_FOLDER = "segmented-solar-dataset"
! mkdir {OUTPUT_FOLDER}

In [31]:
for image_name in dataset:
    # Sirf common image files ko process karein
    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    print(f"Processing image: {image_name}")

    image_path = os.path.join(IMAGE_FOLDER, image_name)
    output_path = os.path.join(OUTPUT_FOLDER, image_name)
    
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    print("Setting image in predictor...")
    predictor.set_image(image_rgb)
    
    # --- Image Load Karein ---
    # OpenCV image BGR format mein parhta hai
    image_bgr = cv2.imread(image_path)
    if image_bgr is None:
        print(f"Warning: Could not read {image_name}. Skipping.")
        continue
        
    # SAM RGB format istemal karta hai
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # --- 4. Segmentation Run Karein ---
    # Yeh woh qadam hai jahan SAM apna jadoo chalata hai
    masks = mask_generator.generate(image_rgb)

    # --- 5. Result Save Karein ---
    # Hum original image par masks ko overlay kar ke save karenge
    plt.figure(figsize=(10, 10))
    plt.imshow(image_rgb)
    show_anns(masks)
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close() # Memory bachane ke liye figure ko band karein

print("All images processed and saved in:", OUTPUT_FOLDER)

Processing image: 31.483888_74.409957_z20_s600_sc2.png
Setting image in predictor...
All images processed and saved in: segmented-solar-dataset


In [ ]:
plt.figure()
plt.imshow(IMAGE_FOLDER + "33.697125_73.021447_z20_s600_sc2.png")